In [0]:
-- Query to read PDFs from the specified path and parse their content using ai_parse_document function
CREATE OR REPLACE TABLE gshen_catalog.yvr_airport.parsed_pdf AS(
SELECT
  path,
  ai_parse_document(content) parsed_content
FROM READ_FILES('/Volumes/gshen_catalog/yvr_airport/pdfs', format => 'binaryFile'));

-- Explode the data into individual rows
SELECT
  path,
  exploded_elements.pos,
  exploded_elements.value:content
FROM gshen_catalog.yvr_airport.parsed_pdf as t,
LATERAL variant_explode_outer(t.parsed_content:document:elements) as exploded_elements

-- Turn it into a table and enable CDC
-- Create a table with one row per PDF element, including file path, element position, and content text
CREATE OR REPLACE TABLE gshen_catalog.yvr_airport.parsed_pdf_chunked AS
(
  SELECT
    path, -- File path of the PDF
    exploded_elements.pos, -- Position of the element within the document
    CAST(exploded_elements.value:content AS STRING) AS content -- Extracted text content of the element
  FROM gshen_catalog.yvr_airport.parsed_pdf AS t,
  LATERAL variant_explode_outer(t.parsed_content:document:elements) AS exploded_elements -- Explode elements array into rows
)

ALTER TABLE `gshen_catalog`.`yvr_airport`.`parsed_pdf_chunked` SET TBLPROPERTIES (delta.enableChangeDataFeed = true)
